In [1]:
import json
import pandas as pd

# Variables

In [2]:
fasta_file_path = './data/genome/gencode.v38.transcripts.fa'
json_geneset_fname = './data/c2.cp.kegg_medicus.v2023.2.Hs.json'

target_gene_sets = ["M47495", "M47605", "M47715"]


out_indexed_gset = f'./output/gene_sets_with_indices__{"_".join(target_gene_sets)}.tsv'


# M47495
"geneSymbols": [
            "ARAF",
            "BRAF",
            "EGFR",
            "EREG",
            "GRB2",
            "HRAS",
            "KRAS",
            "MAP2K1",
            "MAP2K2",
            "MAPK1",
            "MAPK3",
            "NRAS",
            "RAF1",
            "SOS1",
            "SOS2"
        ],


# M47605
"geneSymbols": [
            "ALAD",
            "ALAS1",
            "ALAS2",
            "CPOX",
            "FECH",
            "HMBS",
            "PPOX",
            "UROD",
            "UROS"
        ],


# M47715
"geneSymbols": [
            "ABI1",
            "ACTB",
            "ACTG1",
            "ACTR2",
            "ACTR3",
            "ARF1",
            "ARF6",
            "ARPC1A",
            "ARPC1B",
            "ARPC2",
            "ARPC3",
            "ARPC4",
            "ARPC5",
            "ARPC5L",
            "BRK1",
            "CYFIP1",
            "CYFIP2",
            "CYTH1",
            "CYTH2",
            "CYTH3",
            "CYTH4",
            "NCKAP1",
            "WASF1",
            "WASF2",
            "WASF3"
        ],        

# Extract Target genes with matching index from Fasta file

In [3]:
with open(json_geneset_fname, 'r') as json_file:
    gene_sets = json.load(json_file)

# Extract the gene names from the JSON file - gene sets
gene_set_data = []
for gene_set_name, gene_set_info in gene_sets.items():
    if gene_set_info["systematicName"] in target_gene_sets:
        for gene in gene_set_info["geneSymbols"]:
            gene_set_data.append((gene_set_info["systematicName"], gene))

# Map gene names to their first occurrence index
gene_index_map = {}
with open(fasta_file_path, 'r') as fasta:
    index = 1
    for line in fasta:
        if line.startswith('>'):
            gene_name = line.strip().split('|')[5]
            if gene_name not in gene_index_map:
                gene_index_map[gene_name] = index
            index += 1

# Compare the gene names from the JSON file to the FASTA file
genes_not_found = []
output_data = []
for gene_set_name, gene in gene_set_data:
    if gene in gene_index_map:
        output_data.append((gene_set_name, gene, gene_index_map[gene]))
    else:
        genes_not_found.append(gene)

# Names of genes not found in the FASTA file
if len(genes_not_found) > 0:
    _go = ", ".join(genes_not_found)
    print(f"Genes not found in FASTA file:{_go}")
else:
    print('All genes were found in the reference Fasta file.')

# Write output table: gene set - target gene - index in Fasta
output_df = pd.DataFrame(output_data, columns=['gene_set', 'target_gene', 'index'])
output_df

All genes were found in the reference Fasta file.


,gene_set,target_gene,index
0,M47495,ARAF,230520
1,M47495,BRAF,94630
2,M47495,EGFR,88559
3,M47495,EREG,56700
4,M47495,GRB2,194138
5,M47495,HRAS,122636
6,M47495,KRAS,139341
7,M47495,MAP2K1,167339
8,M47495,MAP2K2,202275
9,M47495,MAPK1,224591


In [4]:
# Verify index
output_df['index'].agg(['min','max'])

min      3556
max    231417
Name: index, dtype: int64

### Filter out shared genes

handle shared genes between gene sets

In [5]:
output_df.shape

(49, 3)

In [6]:
# keep first occurance
df_filtered = output_df.drop_duplicates(subset='target_gene', keep='first')
df_filtered.shape

(49, 3)

### Save resulted table

In [7]:

df_filtered.to_csv(out_indexed_gset, sep='\t', index=False)